In [1]:
# Implied Volatility Finder


In [2]:
from libs import connections


In [3]:
apca_connection = connections.alpaca_trading_connection()
response = apca_connection.get_asset_history('AAPL', '1D', 'day', '1-1-2023', '8-31-2023')
response


TypeError: REST.get_bars() got multiple values for argument 'start'